<a href="https://colab.research.google.com/github/JAlonMor/Webscraping/blob/main/EDA_coches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Clonar repositorio de GitHub si no está disponible

import os
curr_dir = os.getcwd()

if not os.path.exists(os.path.join(curr_dir, '../.ROOT_DIR')):
  !git clone https://github.com/JAlonMor/Webscraping.git
  os.chdir(os.path.join(curr_dir, 'Webscraping/EDA'))



Cloning into 'Webscraping'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 33 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [17]:
# Creo DataFrame a partir de los datos obtenidos mediante web scraping

coches = pd.read_csv("/content/Webscraping/EDA/coches_net.csv")

In [18]:
coches.head()

,titulo,precio,kilometros,año,combustible,ciudad,garantia,url
0,MERCEDES-BENZ Clase C C 220 d 4p.,16200,281000,2017,1.0,Baleares,False,/mercedes-benz-clase-c-c-220-d-4p-diesel-2017-...
1,MERCEDES-BENZ Clase C C 220 d Estate 5p.,26900,114336,2017,1.0,Badajoz,True,/mercedes-benz-clase-c-c-220-d-estate-5p-diese...
2,MERCEDES-BENZ Clase C C 220 d Avantgarde Estat...,11000,287500,2015,1.0,Valladolid,False,/mercedes-benz-clase-c-c-220-d-avantgarde-esta...
3,MERCEDES-BENZ Clase C C 220 d Avantgarde 4p.,22900,110000,2015,1.0,Valencia,True,/mercedes-benz-clase-c-c-220-d-avantgarde-4p-d...
4,MERCEDES-BENZ Clase C C 300 d 4p.,68000,10650,2021,4.0,Cádiz,True,/mercedes-benz-clase-c-c-300-d-4p-electrico-hi...


In [19]:
coches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927 entries, 0 to 1926
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   titulo       1927 non-null   object 
 1   precio       1927 non-null   int64  
 2   kilometros   1927 non-null   int64  
 3   año          1927 non-null   int64  
 4   combustible  1925 non-null   float64
 5   ciudad       1927 non-null   object 
 6   garantia     1927 non-null   bool   
 7   url          1927 non-null   object 
dtypes: bool(1), float64(1), int64(3), object(3)
memory usage: 107.4+ KB


## Limpieza

In [20]:
# Tenemos dos valores nulos en la variable combustible

coches[coches.combustible.isnull()]

,titulo,precio,kilometros,año,combustible,ciudad,garantia,url
416,MERCEDES-BENZ Clase C 5p.,28500,150000,2015,NaN,Guipúzcoa,False,/mercedes-benz-clase-c--5p-otros-2015-en-guipu...
1566,MERCEDES-BENZ,33300,34400,2017,NaN,Murcia,False,/mercedes-benz-clase-c--5p-otros-2017-en-murci...


In [21]:
# Al ser solo dos observaciones, trataré de solucionarlo manualmente. Veamos las URL

coches.iloc[416,:].url

'/mercedes-benz-clase-c--5p-otros-2015-en-guipuzcoa-51760617-covo.aspx'

Es un híbrido diésel (4)

In [22]:
coches.iloc[1566,:].url

'/mercedes-benz-clase-c--5p-otros-2017-en-murcia-51484724-covo.aspx'

Es un híbrido enchufable (gasolina) (5)

In [23]:
coches.combustible.unique()

array([ 1.,  4.,  2.,  5., nan])

Códigos de los diferentes combustibles

* 1 $\rightarrow$ Diésel
* 2 $\rightarrow$ Gasolina
* 4 $\rightarrow$ Híbrido
* 5 $\rightarrow$ Híbrido enchufable


In [24]:
coches.iloc[416,:].combustible = 4.0
coches.iloc[1566,:].combustible = 5.0

In [25]:
coches.iloc[416,4] = 4.0
coches.iloc[1566,4] = 5.0

In [26]:
coches[coches.combustible.isnull()]

,titulo,precio,kilometros,año,combustible,ciudad,garantia,url


In [27]:
# Ahora cambio el tipo de variable a entero (combustible)
coches['combustible'] = coches['combustible'].astype(int)
coches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927 entries, 0 to 1926
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   titulo       1927 non-null   object
 1   precio       1927 non-null   int64 
 2   kilometros   1927 non-null   int64 
 3   año          1927 non-null   int64 
 4   combustible  1927 non-null   int64 
 5   ciudad       1927 non-null   object
 6   garantia     1927 non-null   bool  
 7   url          1927 non-null   object
dtypes: bool(1), int64(4), object(3)
memory usage: 107.4+ KB


In [28]:
coches.head()

,titulo,precio,kilometros,año,combustible,ciudad,garantia,url
0,MERCEDES-BENZ Clase C C 220 d 4p.,16200,281000,2017,1,Baleares,False,/mercedes-benz-clase-c-c-220-d-4p-diesel-2017-...
1,MERCEDES-BENZ Clase C C 220 d Estate 5p.,26900,114336,2017,1,Badajoz,True,/mercedes-benz-clase-c-c-220-d-estate-5p-diese...
2,MERCEDES-BENZ Clase C C 220 d Avantgarde Estat...,11000,287500,2015,1,Valladolid,False,/mercedes-benz-clase-c-c-220-d-avantgarde-esta...
3,MERCEDES-BENZ Clase C C 220 d Avantgarde 4p.,22900,110000,2015,1,Valencia,True,/mercedes-benz-clase-c-c-220-d-avantgarde-4p-d...
4,MERCEDES-BENZ Clase C C 300 d 4p.,68000,10650,2021,4,Cádiz,True,/mercedes-benz-clase-c-c-300-d-4p-electrico-hi...


## Transformaciones

Transformaciones a realizar:

Sacar más información de las columnas "titulo" y "url"
Crear nueva columna con url completa añadiendo la url base de coches.net
Crear otra variable con las coordenadas de cada ciudad
Crear variable con max-min de kilometros (el escalado se hace con los datos de entrenamiento creo)
Crear variable con la antigüedad de cada coche: now-año
Crear variable dummy con el combustible

Buscar outliers en las variables, boxplot
Realizar estudios de distribuciones de las variables
Estudio de la correlaciones entre las variables.
Estudio de las correlaciones con la variable objetivo "precio"
Hacer gráfico con mapa con las posiciones de las ciudades donde se venden los coches.
Usar métodos de filtrado para regesion, f_regression y mutual_info_regession. F-test score y Mutual information




### TRANSFORMACIONES<br/>



* [X] Poner URL completa
* [X] Extraer más información de "titulo" y "url"
* [ ] Coordenadas GPS de ciudades
* [ ] Escalado de variable kilómetros
* [ ] Crear nueva variable con la antigüedad del coches en años. Escalar.
* [ ] Variables binarias para combustible
* [ ] Categorical type will be better in 'name'column.



### EXPLORACIONES

* [ ] Box plot de caracterísitcas.
* [ ] Análisis por agrupación.
* [ ] Estudio de las distintas distribuciones.
* [ ] Estudio de correlaciones entre las variables.
* [ ] Estudio de correlaciones entre las variables y "precio".
* [ ] Estudio según localización GPS.

### Poner URL completa

In [41]:
# Creamos URL completa
from urllib.parse import urljoin

url_base = "https://www.coches.net/"
coches['web'] = [urljoin(url_base, url) for url in coches['url']]

In [43]:
# Compruebo
coches.web[0]

'https://www.coches.net/mercedes-benz-clase-c-c-220-d-4p-diesel-2017-en-baleares-51618684-covo.aspx'

### Extraer información de "titulo" y de "url"

In [55]:
# Aumento la cantidad de caracteres a mostrar por pantalla
pd.options.display.max_colwidth = 200

In [56]:
print(coches.loc[[0],['titulo']])
print(coches.loc[[0],['url']]) ##, 'url']]),

                              titulo
0  MERCEDES-BENZ Clase C C 220 d 4p.
                                                                            url
0  /mercedes-benz-clase-c-c-220-d-4p-diesel-2017-en-baleares-51618684-covo.aspx


In [79]:
coches.sample(5)

,titulo,precio,kilometros,año,combustible,ciudad,garantia,url,web,tipo
1727,MERCEDES-BENZ Clase C MercedesAMG C 43 4MATIC 4p.,44500,99950,2017,2,Málaga,False,/mercedes-benz-clase-c-mercedesamg-c-43-4matic-4p-gasolina-2017-en-malaga-51293949-covo.aspx,https://www.coches.net/mercedes-benz-clase-c-mercedesamg-c-43-4matic-4p-gasolina-2017-en-malaga-51293949-covo.aspx,43
1187,MERCEDES-BENZ Clase C C 300 h Estate 5p.,27990,97000,2016,4,Sevilla,True,/mercedes-benz-clase-c-c-300-h-estate-5p-electrico-hibrido-2016-en-sevilla-51632565-covo.aspx,https://www.coches.net/mercedes-benz-clase-c-c-300-h-estate-5p-electrico-hibrido-2016-en-sevilla-51632565-covo.aspx,300
318,MERCEDES-BENZ Clase C C 220 d 4MATIC 4p.,29900,49500,2017,1,Madrid,False,/mercedes-benz-clase-c-c-220-d-4matic-4p-diesel-2017-en-madrid-51767434-covo.aspx,https://www.coches.net/mercedes-benz-clase-c-c-220-d-4matic-4p-diesel-2017-en-madrid-51767434-covo.aspx,220
1515,MERCEDES-BENZ Clase C Coupe C 220 d 2p.,43900,23400,2019,1,Madrid,True,/mercedes-benz-clase-c-coupe-c-220-d-2p-diesel-2019-en-madrid-50397184-covo.aspx,https://www.coches.net/mercedes-benz-clase-c-coupe-c-220-d-2p-diesel-2019-en-madrid-50397184-covo.aspx,220
1905,MERCEDES-BENZ Clase C C 220 d AMG Line 4p.,26995,40494,2016,1,Madrid,False,/mercedes-benz-clase-c-c-220-d-amg-line-4p-diesel-2016-en-madrid-44516841-covo.aspx,https://www.coches.net/mercedes-benz-clase-c-c-220-d-amg-line-4p-diesel-2016-en-madrid-44516841-covo.aspx,220


In [59]:
import regex as re

Crear nueva variable "tipo", con tipo de modelo clase C que es el coche. En caso no obtener esa información, identificar con la palabra "NoInfo"

In [74]:
coches['tipo'] = coches['url'].str.extract('-(\d{2,3})-').fillna('NoInfo')

In [76]:
coches.tipo.sample(10)

1803    220
746     220
1858    220
386     220
915     220
1483    200
346     220
154     200
674     200
1211    220
Name: tipo, dtype: object

In [78]:
coches.value_counts('tipo')

tipo
220       1092
200        279
NoInfo     132
250        111
63          76
300         71
43          69
180         56
350         33
450          4
400          3
45           1
dtype: int64

Crear nueva variable booleana, indicango si el vehículo tiene el acabado AMG True o no False

In [83]:
coches['AMG'] = False
coches.loc[coches.url.str.contains('amg', case=True),'AMG'] = True

In [86]:
coches.value_counts('AMG')

AMG
False    1521
True      406
dtype: int64

### Coordenadas GPS de ciudades

In [88]:
from geopy.geocoders import Nominatim

In [89]:
## https://www.tutorialspoint.com/how-to-get-the-longitude-and-latitude-of-a-city-using-python#:~:text=Use%20the%20geolocator.,state%2C%20and%20country%20using%20address.